In [18]:
import os
import subprocess
import re

def searchFile(ROOT_PATH,saveFile):
    if not os.path.isdir(os.path.dirname(saveFile)):
        os.makedirs(os.path.dirname(saveFile))
    allFile={}
    count=0
    for pathname,dirnames,filenames in os.walk(ROOT_PATH):
        for filename in filenames:
            if filename.endswith(".java"):
                fullPath=os.path.join(pathname,filename)
                count=count+1
                if "info" not in fullPath:
                    parentFolder=re.sub(r"\\src\\.*","",fullPath)
                    if parentFolder in allFile:
                        allFile[parentFolder].append(fullPath)
                    else:
                        allFile.setdefault(parentFolder,[fullPath])
                        
    with open(saveFile,"w",encoding="utf-8") as f:              
        for key,item in allFile.items():
            for pathfile in item:
                print(pathfile,file=f)
            print(file=f)
    print(count)
    
#searchFile(os.path.join("C:\\Users\\syuuj","gitProject\\CoreNLP"),os.path.join("C:\\Users\\syuuj","getData_python\\full.txt"))

In [23]:
import pandas as pd

def getCodeSmellClass():
    dfGod=pd.read_csv("ground truth\\GodClass.csv")
    dfRefused=pd.read_csv("ground truth\\RefusedBequest.csv")
    dfGod=dfGod[["nomeSistema","class"]]
    dfRefused=dfRefused[["nomeSistema","class"]]
    allClass=pd.concat([dfGod,dfRefused])
    allClass=allClass.drop_duplicates()
    return allClass
   
RootFolder="C:\\Users\\syuuj"
smellFileName="dbeaver"

saveFile=os.path.join(RootFolder,"getData_python\\full.txt")
project=os.path.join(RootFolder,"gitProject","dbeaver")            #ここを変更
saveFolder=os.path.join(RootFolder,"data",os.path.basename(project))
if not os.path.isdir(saveFolder):
    os.makedirs(saveFolder)

checkoutVersions=["21.0.2","21.0.1"]                                        #ここを変更

CKAnalyze=["java","-jar","ck-0.7.1-SNAPSHOT-jar-with-dependencies.jar",project,"False","0","True",saveFolder]
SpoonAnalyze=["java","-jar","demo-1.0-snapshot.jar",saveFile,saveFolder]

i=1
for version in checkoutVersions:
    gitCheckout=["git","checkout",version]
    subprocess.run(gitCheckout,cwd=project,check=True)
    searchFile(project,saveFile)
    saveMetricsFile=os.path.join(saveFolder,str(i))
    CKAnalyze[7]=saveMetricsFile
    SpoonAnalyze[4]=saveMetricsFile
    subprocess.run(CKAnalyze,cwd=os.path.join(RootFolder,"ck\\target"),check=True)
    if i==1:
        df=pd.read_csv(saveMetricsFile+"class.csv",encoding="cp932")
        allClass=getCodeSmellClass()
        group=allClass.groupby("nomeSistema")
        classes=group.get_group(smellFileName)
        print(len(set(df['class']) & set(classes['class'])),"/",len(classes))
        data=(set(df['class']) & set(classes['class']))
        print(classes[~classes['class'].isin(df['class'])])

    #subprocess.run(SpoonAnalyze,cwd=os.path.join(os.getcwd(),"spoon\\target"),check=True)    
    i=i+1

4156
115 / 116
    nomeSistema                                              class
408     dbeaver  org.jkiss.dbeaver.ui.editors.sql.SQLEditor.Que...
4141


In [20]:
import pandas as pd
dfGod=pd.read_csv("ground truth\\GodClass.csv")
dfRefused=pd.read_csv("ground truth\\RefusedBequest.csv")
dfGod=dfGod[["nomeSistema","class"]]
dfRefused=dfRefused[["nomeSistema","class"]]
allClass=pd.concat([dfGod,dfRefused])


In [21]:
group=allClass.groupby("nomeSistema")
classes=group.get_group("checkstyle")
print(len(classes))

14
